## 收集

In [ ]:
import pandas as pd

In [ ]:
patients = pd.read_csv('patients.csv')
treatments = pd.read_csv('treatments.csv')
adverse_reactions = pd.read_csv('adverse_reactions.csv')

## 评估

In [ ]:
patients

<div>
<style>
    .dataframe thead tr:only-child th {
        text-align: right;
    }

In [ ]:
treatments

<div>
<style>
    .dataframe thead tr:only-child th {
        text-align: right;
    }

In [ ]:
adverse_reactions

<div>
<style>
    .dataframe thead tr:only-child th {
        text-align: right;
    }

In [ ]:
patients.info()

In [ ]:
treatments.info()

In [ ]:
adverse_reactions.info()

In [ ]:
all_columns = pd.Series(list(patients) + list(treatments) + list(adverse_reactions))
all_columns[all_columns.duplicated()]

In [ ]:
list(patients)

In [ ]:
patients[patients['address'].isnull()]

<div>
<style>
    .dataframe thead tr:only-child th {
        text-align: right;
    }

In [ ]:
patients.describe()

<div>
<style>
    .dataframe thead tr:only-child th {
        text-align: right;
    }

In [ ]:
treatments.describe()

<div>
<style>
    .dataframe thead tr:only-child th {
        text-align: right;
    }

In [ ]:
patients.sample(5)

<div>
<style>
    .dataframe thead tr:only-child th {
        text-align: right;
    }

In [ ]:
patients.surname.value_counts()

In [ ]:
patients.address.value_counts()

In [ ]:
patients[patients.address.duplicated()]

<div>
<style>
    .dataframe thead tr:only-child th {
        text-align: right;
    }

In [ ]:
patients.weight.sort_values()

In [ ]:
weight_lbs = patients[patients.surname == 'Zaitseva'].weight * 2.20462
height_in = patients[patients.surname == 'Zaitseva'].height
bmi_check = 703 * weight_lbs / (height_in * height_in)
bmi_check

In [ ]:
patients[patients.surname == 'Zaitseva'].bmi

In [ ]:
sum(treatments.auralin.isnull())

In [ ]:
sum(treatments.novodra.isnull())

#### 质量
##### `patients` 表格
- 邮编是浮点，而不是字符串
- 邮编有时是四位数
- Tim Neudorf 的身高是 27 英尺，而不是 72 英尺
- 有时使用州的全称，其他用简称
- Dsvid Gustafsson
- 缺少人口统计信息 (地址列至联系方式列) ***(无法清理)***
- 错误的数据类型 (指定性别、州、邮编和出生日期列)
- 多个手机号格式
- John Doe 的默认数据
- Jakobsen、Gersten、Taylor 多条记录
- Zaitseva 体重单位是 kgs，不是 lbs 

##### `treatments` 表格
- 缺少 HbA1c 变化
- Auralin 和 Novodra 初始剂量和最终剂量中的字母 'u' 
- 小写形式的姓和名
- 缺少记录 (280 而不是 350)
- 错误的数据类型 (auralin 和 novodra 列)
- HbA1c 变化不准确 (首个 4s 误写为 9s)
- auralin 和 novodra 列中空值用破折号 (-) 表示

##### `adverse_reactions` 表格
- 小写形式的姓和名

#### 整洁度
-  `patients` 表格中的联系方式一列应该拆分为手机号和邮箱两列
-  `treatments` 表格两列中的三个变量 (治疗、初始剂量和最终剂量)
-  不良反应应是 `treatments` 表格的一部分
-  `patients` 表格中的姓和名两列复制到  `treatments` 和 `adverse_reactions` 表格中

## 清理

In [ ]:
patients_clean = patients.copy()
treatments_clean = treatments.copy()
adverse_reactions_clean = adverse_reactions.copy()

### 缺失数据

#### `treatments`：缺少记录 (280 而不是 350)

##### 定义
将切口治疗导入一个 DataFrame 中，并它拼接到初始治疗 DataFrame  中。

##### 代码

In [ ]:
treatments_cut = pd.read_csv('treatments_cut.csv')
treatments_clean = pd.concat([treatments_clean, treatments_cut],
                             ignore_index=True)

##### 测试

In [ ]:
treatments_clean.head()

<div>
<style>
    .dataframe thead tr:only-child th {
        text-align: right;
    }

In [ ]:
treatments_clean.tail()

<div>
<style>
    .dataframe thead tr:only-child th {
        text-align: right;
    }

#### `treatments`：缺少 HbA1c 变化和 HbA1c 变化不准确 (首个 4s 误写为 9s) 

##### 定义
重新计算 `hba1c_change` 一列： `hba1c_start` 减去 `hba1c_end`。

##### 代码

In [ ]:
treatments_clean.hba1c_change = (treatments_clean.hba1c_start - 
                                 treatments_clean.hba1c_end)

##### Test

In [ ]:
treatments_clean.hba1c_change.head()

### 整洁度

####  `patients` 表格中的联系方式一列包括两个变量：手机号和邮箱

##### 定义
使用正则表达式和 pandas 的 `str.extract` 方法，从 *联系方式* 一列提取变量*手机号* 和 *邮箱* 。完成后删除 *联系方式* 一列。

##### 代码

In [ ]:
patients_clean['phone_number'] = patients_clean.contact.str.extract('((?:\+?\d{1,2}\s)?\(?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4})', expand=True)

# [a-zA-Z] 表明数据集中的邮箱开头和结尾都是字母
patients_clean['email'] = patients_clean.contact.str.extract('([a-zA-Z][a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+[a-zA-Z])', expand=True)

# Note: axis=1 表明我们参考的是一列，而不是一行
patients_clean = patients_clean.drop('contact', axis=1)

##### 测试

In [ ]:
# 确认联系方式一列可以运行
list(patients_clean)

In [ ]:
patients_clean.phone_number.sample(25)

In [ ]:
# 确认没有邮箱以整数开头 (正则表达式不能匹配整数)
patients_clean.email.sort_values().head()

#### `treatments` 表格两列中的三个变量 (治疗、初始剂量和最终剂量)

##### 定义
在 *auralin* 和 *novodra* 两列中加入 *治疗* 和 *剂量* 列 (这时剂量仍然包含初始剂量和最终剂量)。然后使用破折号 ' - ' 把剂量列切分，得到 *初始剂量* 和 *最终剂量* 两列。删除中间使用的 *剂量* 一列。

##### 代码

In [ ]:
treatments_clean = pd.melt(treatments_clean, id_vars=['given_name', 'surname', 'hba1c_start', 'hba1c_end', 'hba1c_change'],
                           var_name='treatment', value_name='dose')
treatments_clean = treatments_clean[treatments_clean.dose != "-"]
treatments_clean['dose_start'], treatments_clean['dose_end'] = treatments_clean['dose'].str.split(' - ', 1).str
treatments_clean = treatments_clean.drop('dose', axis=1)

##### 测试

In [ ]:
treatments_clean.head()

<div>
<style>
    .dataframe thead tr:only-child th {
        text-align: right;
    }

#### 不良反应应是 `treatments` 表格的一部分

##### 定义
将 *不良反应* 一列合并到 `treatments` 表格中，按照 *名* 和 *姓*进行合并。

##### 代码

In [ ]:
treatments_clean = pd.merge(treatments_clean, adverse_reactions_clean,
                            on=['given_name', 'surname'], how='left')

##### 测试

In [ ]:
treatments_clean

<div>
<style>
    .dataframe thead tr:only-child th {
        text-align: right;
    }

####  `patients` 表格中的姓和名两列复制到 `treatments` 和 `adverse_reactions`表格中，以及小写形式的姓和名 

##### 定义
我们不再需要不良反应表格，所以忽略这部分内容。在 `patients` 表格中，提取病人编号和姓名，然后把这些名字改为小写形式，加入到 `treatments` 表格中。然后删除治疗表格中的姓和名两列 (这样小写形式不再是问题了)。

##### 代码

In [ ]:
id_names = patients_clean[['patient_id', 'given_name', 'surname']]
id_names.given_name = id_names.given_name.str.lower()
id_names.surname = id_names.surname.str.lower()
treatments_clean = pd.merge(treatments_clean, id_names, on=['given_name', 'surname'])
treatments_clean = treatments_clean.drop(['given_name', 'surname'], axis=1)

##### 测试

In [ ]:
# 确认进行了正确融合
treatments_clean

<div>
<style>
    .dataframe thead tr:only-child th {
        text-align: right;
    }

In [ ]:
# 病人编号应该是唯一复制的一列
all_columns = pd.Series(list(patients_clean) + list(treatments_clean))
all_columns[all_columns.duplicated()]